In [ ]:
import os
import cv2
import random
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
def train_val_split(dataset_folder, output_folder, val_split, apply_aug=True):

    for folder in [output_folder, os.path.join(output_folder, 'training'), os.path.join(output_folder, 'validation')]:
        if not os.path.exists(folder):
            os.makedirs(folder)

    # Parameter untuk ukuran dataset validation
    validation_split = val_split

    # apply_augmentation
    def apply_augmentation(image):
        
        if random.random() < 0.5:
            # Flip gambar secara horizontal
            image = cv2.flip(image, 1)  

        if random.random() < 0.5:
            # Menaikkan dan menurunkan kecerahan
            alpha = random.uniform(0.7, 1.3)
            image = cv2.multiply(image, np.array([alpha]))

        if random.random() < 0.5:
            # Menaikkan dan menurunkan kontras
            alpha = random.uniform(0.7, 1.3)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            gray = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
            image = cv2.addWeighted(image, alpha, gray, 1 - alpha, 0)

        if random.random() < 0.5:
            # Merubah kecerahan dan kontras secara acak
            alpha = random.uniform(0.7, 1.3)
            image = cv2.multiply(image, np.array([alpha]))

            alpha = random.uniform(0.7, 1.3)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            gray = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
            image = cv2.addWeighted(image, alpha, gray, 1 - alpha, 0)

        if random.random() < 0.5:
            # Rotasi 90 derajat
            image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)

        return image

    for class_name in os.listdir(dataset_folder):
        class_folder = os.path.join(dataset_folder, class_name)

        training_class_folder = os.path.join(output_folder, 'training', class_name)
        validation_class_folder = os.path.join(output_folder, 'validation', class_name)
        for folder in [training_class_folder, validation_class_folder]:
            if not os.path.exists(folder):
                os.makedirs(folder)

        image_files = os.listdir(class_folder)
        random.shuffle(image_files)
        num_validation_images = int(len(image_files) * validation_split)

        for i, image_name in enumerate(image_files):
            image_path = os.path.join(class_folder, image_name)

            if i < num_validation_images:
                output_image_path = os.path.join(validation_class_folder, image_name)
            else:
                output_image_path = os.path.join(training_class_folder, image_name)

            image = cv2.imread(image_path)

            if apply_aug == True:
                image = apply_augmentation(image)
            else:
                image = image
                
            cv2.imwrite(output_image_path, image)

In [ ]:
def datagen(input_directory, input_img_size: int, mode_kelas="sparse"):

    data_generator = ImageDataGenerator(rescale=1/255,
                              rotation_range=40,
                              width_shift_range=0.2,
                              height_shift_range=0.2,
                              shear_range=0.2,
                              zoom_range=0.2,
                              horizontal_flip=True,
                              fill_mode='nearest')
    
    output = data_generator.flow_from_directory(input_directory, (input_img_size, input_img_size), class_mode=mode_kelas)

    return output

In [ ]:
def write_image(path, image, filename):
  if not os.path.exists(path):
    os.makedirs(path)
  path = os.path.join(path, str(filename))
  cv2.imwrite(path, image)

In [ ]:
def detect_and_save_faces(image_path, dataset_path):
    # Load cascade classifier untuk deteksi wajah
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Baca gambar
    image = cv2.imread(image_path)
    image_name = os.path.basename(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Deteksi wajah pada gambar
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Simpan area wajah yang terdeteksi
    if len(faces)>0:
        for (x, y, w, h) in faces:
              cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 0)
              crop = image[y:y + h, x:x + w]
              output_image_path_training = os.path.join(dataset_path, "dataset")
              write_image(output_image_path_training, crop, image_name)
    else:
        print('Wajah Tidak Terdeteksi', image_name )

In [ ]:
def file_iteration(rawdata_path, dataset_path):
  for files in os.listdir(rawdata_path):
    if files.lower().endswith(('.png', '.jpg', '.jpeg')):
        input_image_path = os.path.join(rawdata_path, files)
        detect_and_save_faces(input_image_path, dataset_path)